### This is a Multi-class Classification Model using Cohere.ai 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install cohere
!pip install -U featuretools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cohere: filename=cohere-2.9.1-cp38-cp38-linux_x86_64.whl size=9627 sha256=5e946327a22d89082c438448193737daab73ba2f649f1d2376ff03a89d62379b
  Stored in directory: /root/.cache/pip/wheels/c8/79/5b/36adcce533f4b0ed540beef7e6ea61d637a10cb2c8d4670153
Successfully built cohere
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 13.6 MB/s 
     |████████████████████████████████| 229 kB 78.8 MB/s 
     |████████████████████████████████| 12.2 MB 61.8 MB/s 
     |████████████████████████████████| 280 kB 73.4 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfull

In [ ]:
import featuretools

In [ ]:
import os
import cohere
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
pd.set_option('display.max_colwidth', None)


In [ ]:
#read csv file 
wd = "/content/drive/Shareddrives/UglySweater"
os.chdir(wd)
df = pd.read_csv(f"{wd}/preprocessed.csv",header=0)
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.dropna()
df = df.reset_index()
df = df.drop(['index'], axis=1)
df.head(5)

,posts,type,multimedia
0,♪ tails paper triggers past,INFJ,website
1,enfp intj moments sportscenter top ten plays pranks oh stop stop oh many people want early good morning goodnight guys already discussed since saying rooms im certainly glad didnt get outfit last night hey alex almost ready weve gotta go yeah almost ready alex alex fabulous alex always ready um almost im sorry like five minutes dont dont nope would got dress living room oh took earlier dont want weigh know put outfit night like put night wouldnt problem live seattle january hope sometimes okay im ready im ready need jacket warm one waterproofs right doesnt really match outfit oh well right coat ill put screen oh thank im going put hat,INFJ,website
2,lifechanging experience life,INFJ,reg_text
3,repeat today faster slow show theeres corner anarwon,INFJ,video
4,may perc experience immerse,INFJ,reg_text


In [ ]:
df_filter=df[df['multimedia']=='reg_text']

In [ ]:
df_filter

,posts,type,multimedia
2,lifechanging experience life,INFJ,reg_text
4,may perc experience immerse,INFJ,reg_text
6,hello enfj sorry hear distress natural relationship perfection time every moment existence try figure hard times times growth,INFJ,reg_text
7,welcome stuff,INFJ,reg_text
9,prozac wellbrutin least thirty minutes moving legs dont mean moving sitting desk chair weed moderation maybe try edibles healthier alternative,INFJ,reg_text
...,...,...,...
404312,going close facebook months back well wanting able message family ausse school friends found connected websites,INFP,reg_text
404313,seconds mars collections seems fitting mood right,INFP,reg_text
404314,seen agree actually think first time watched movie beginning got powers kinda thought andrew would never work right,INFP,reg_text
404315,ok watched underworld awakening must say really good film compared films last months anyway dont think good first,INFP,reg_text


In [ ]:
df_filter.shape

(394043, 3)

### Balanced Data

In [ ]:
df['type'].value_counts()

INFP    85323
INFJ    68953
INTP    60346
INTJ    50312
ENTP    32629
ENFP    31601
ISTP    15649
ISFP    12269
ENTJ    10872
ISTJ     9495
ENFJ     8999
ISFJ     7746
ESTP     4160
ESFP     2114
ESFJ     1986
ESTJ     1863
Name: type, dtype: int64

In [ ]:
# num_examples = 500  
# df = df.head(num_examples)

In [ ]:
df_filter['type'].value_counts()

INFP    83073
INFJ    67250
INTP    58686
INTJ    49035
ENTP    31885
ENFP    30943
ISTP    15213
ISFP    11856
ENTJ    10633
ISTJ     9240
ENFJ     8779
ISFJ     7538
ESTP     4058
ESFP     2075
ESFJ     1964
ESTJ     1815
Name: type, dtype: int64

In [ ]:
def passMin(type_code): 
  return df_filter[df_filter['type'] ==type_code].head(1000).copy()

In [ ]:
balanced_data = pd.DataFrame()
type_codes = df_filter['type'].unique()
for type_code in type_codes[:16]: 
  balanced_data = balanced_data.append([passMin(type_code)])
balanced_data['type'].value_counts()

<ipython-input-12-7660da806dbf>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_data = balanced_data.append([passMin(type_code)])
<ipython-input-12-7660da806dbf>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_data = balanced_data.append([passMin(type_code)])
<ipython-input-12-7660da806dbf>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_data = balanced_data.append([passMin(type_code)])
<ipython-input-12-7660da806dbf>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_data = balanced_data.append([passMin(type_code)])
<ipython-input-12-7660da806dbf>:4: FutureWarning: The frame.append method is depreca

INFJ    1000
ENTP    1000
INTP    1000
INTJ    1000
ENTJ    1000
ENFJ    1000
INFP    1000
ENFP    1000
ISFP    1000
ISTP    1000
ISFJ    1000
ISTJ    1000
ESTP    1000
ESFP    1000
ESTJ    1000
ESFJ    1000
Name: type, dtype: int64

In [ ]:
balanced_data.head(5)

,posts,type,multimedia
2,lifechanging experience life,INFJ,reg_text
4,may perc experience immerse,INFJ,reg_text
6,hello enfj sorry hear distress natural relationship perfection time every moment existence try figure hard times times growth,INFJ,reg_text
7,welcome stuff,INFJ,reg_text
9,prozac wellbrutin least thirty minutes moving legs dont mean moving sitting desk chair weed moderation maybe try edibles healthier alternative,INFJ,reg_text


### Encode

In [ ]:
label = LabelEncoder()
balanced_data['label'] = label.fit_transform(balanced_data['type'])
balanced_data.head(5)

,posts,type,multimedia,label
2,lifechanging experience life,INFJ,reg_text,8
4,may perc experience immerse,INFJ,reg_text,8
6,hello enfj sorry hear distress natural relationship perfection time every moment existence try figure hard times times growth,INFJ,reg_text,8
7,welcome stuff,INFJ,reg_text,8
9,prozac wellbrutin least thirty minutes moving legs dont mean moving sitting desk chair weed moderation maybe try edibles healthier alternative,INFJ,reg_text,8


### Prepare training and testing sets

In [ ]:
balanced_data['label'].value_counts()

8     1000
3     1000
11    1000
10    1000
2     1000
0     1000
9     1000
1     1000
13    1000
15    1000
12    1000
14    1000
7     1000
5     1000
6     1000
4     1000
Name: label, dtype: int64

In [ ]:
X = balanced_data['posts']
y = balanced_data['label']

In [ ]:
X

2                                                                                                                           lifechanging experience life
4                                                                                                                            may perc experience immerse
6                          hello enfj sorry hear distress natural relationship perfection time every moment existence try figure hard times times growth
7                                                                                                                                          welcome stuff
9         prozac wellbrutin least thirty minutes moving legs dont mean moving sitting desk chair weed moderation maybe try edibles healthier alternative
                                                                               ...                                                                      
222994                                     type chad kelsi taylor think troy esfj 

In [ ]:
#balanced_data['posts'] = balanced_data['posts'].apply(lambda x: x[:100])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(list(X), list(y), test_size = 0.25, random_state = 0)

In [ ]:
print(len(X_train))
print(len(X_test))

12000
4000


### Get the embeddings of the posts 

In [ ]:
# ADD YOUR API KEY HERE
api_key = "7mHjN9V4dW4djta6LdGoXFXlpMwScsIuPdYK6URf"

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [ ]:
# Embed the training set
import time 
embeddings_tests = [] 
embeddings_trains = []
try_train = []
try_test = []
copy_X_train = X_train 
copy_X_test = X_test
for i in range(24): 
  try_train = copy_X_train[:500]
  embeddings_trains.append(co.embed(texts=try_train,
                              model="large",
                              truncate="LEFT").embeddings)
  
  copy_X_train = copy_X_train[500:]
 
  if i <= 19: #19
    try_test = copy_X_test[:200]
    embeddings_tests.append(co.embed(texts=try_test,
                                model="large",
                                truncate="LEFT").embeddings) 
    copy_X_test = copy_X_test[200:]
  time.sleep(60) #set a minute

In [ ]:
embeddings_train = [elem for twod in embeddings_trains for elem in twod]
embeddings_test = [elem for twod in embeddings_tests for elem in twod]
print((len(embeddings_train)))
print(len(embeddings_test))

12000
4000


In [ ]:
#print(f"Review text: {X_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Embedding vector: [0.49047852, -0.18029785, 1.2509766, -0.30639648, -0.9399414, 0.3347168, 1.3789062, 0.31298828, 0.85009766, -1.5380859]


### Train a classifier using the training set

In [ ]:
# import SVM classifier code
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 
svm_classifier.fit(embeddings_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced'))])

In [ ]:

# get the score from the test set, and print it out to screen!
score = svm_classifier.score(embeddings_test, y_test)
print(f"Validation accuracy on Large is {100*score}%!")


Validation accuracy on Large is 18.925%!


In [ ]:
import joblib

In [ ]:
filename = "Completed_model.joblib"
joblib.dump(svm_classifier, filename)

['Completed_model.joblib']

In [ ]:
loaded_model = joblib.load(filename)
result = loaded_model.score(embeddings_test, y_test)
print(result)

0.18925


**Predict**


In [36]:
texts=[" answer things adventures seem ne definitely fe extroverted qualities"]
response = co.embed(texts)
print('Embeddings: {}'.format(response.embeddings))

Embeddings: [[0.5253906, -0.5864258, -1.0371094, 0.3317871, -0.34033203, -1.4882812, -0.16992188, -0.5136719, 0.014694214, -1.7666016, 0.6308594, 2.4121094, -0.31469727, 0.7949219, 0.7841797, 1.6962891, -1.40625, 0.5703125, -1.4228516, -0.8676758, -2.3789062, 0.57373047, 2.2226562, -0.77685547, -2.0800781, -3.3945312, -1.8583984, 2.1230469, -1.0585938, -0.9560547, -0.2783203, 0.07720947, -1.2744141, 0.20983887, 1.2333984, 1.3017578, 0.68310547, 0.66845703, -1.0625, 1.5878906, 2.5292969, -1.5839844, 0.6557617, -0.24572754, -1.2363281, 2.5136719, 0.06890869, -2.8203125, -1.4599609, 1.0244141, 1.8251953, 0.6269531, 0.38452148, -1.0830078, 2.5195312, -0.6503906, 1.4443359, -0.5317383, -2.7578125, -0.30126953, -0.6542969, -0.7675781, 0.04647827, -1.0566406, -0.0010433197, 1.1533203, -0.32885742, -0.53466797, -0.06719971, 1.640625, -0.2734375, 2.0546875, 2.3046875, 0.8925781, 0.66845703, -0.6879883, 2.4296875, -0.8364258, 0.34643555, 2.3203125, 0.17211914, -0.9448242, -1.4902344, 2.28125, -1

In [41]:
response.embeddings[0]

[0.5253906,
 -0.5864258,
 -1.0371094,
 0.3317871,
 -0.34033203,
 -1.4882812,
 -0.16992188,
 -0.5136719,
 0.014694214,
 -1.7666016,
 0.6308594,
 2.4121094,
 -0.31469727,
 0.7949219,
 0.7841797,
 1.6962891,
 -1.40625,
 0.5703125,
 -1.4228516,
 -0.8676758,
 -2.3789062,
 0.57373047,
 2.2226562,
 -0.77685547,
 -2.0800781,
 -3.3945312,
 -1.8583984,
 2.1230469,
 -1.0585938,
 -0.9560547,
 -0.2783203,
 0.07720947,
 -1.2744141,
 0.20983887,
 1.2333984,
 1.3017578,
 0.68310547,
 0.66845703,
 -1.0625,
 1.5878906,
 2.5292969,
 -1.5839844,
 0.6557617,
 -0.24572754,
 -1.2363281,
 2.5136719,
 0.06890869,
 -2.8203125,
 -1.4599609,
 1.0244141,
 1.8251953,
 0.6269531,
 0.38452148,
 -1.0830078,
 2.5195312,
 -0.6503906,
 1.4443359,
 -0.5317383,
 -2.7578125,
 -0.30126953,
 -0.6542969,
 -0.7675781,
 0.04647827,
 -1.0566406,
 -0.0010433197,
 1.1533203,
 -0.32885742,
 -0.53466797,
 -0.06719971,
 1.640625,
 -0.2734375,
 2.0546875,
 2.3046875,
 0.8925781,
 0.66845703,
 -0.6879883,
 2.4296875,
 -0.8364258,
 0.346

In [46]:
got = loaded_model.predict(response.embeddings)

In [49]:
got[0]

4

In [56]:
list1=['ISTJ', 'ISTP', 'ISFJ','ISFP','INFJ','INFP','INTJ','INTP','ESTP','ESTJ','ESFP','ESFJ','ENFP','ENFJ', 
    'ENTP','ENTJ' ]

In [57]:
len(list1)

16

In [59]:
list1[4]

'INFJ'

In [53]:
personality['ISFP']

'The Artist- Easy-going and flexible, they tend to be reserved and artistic.'

### Keras 

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf 

In [ ]:
def create_model(my_learning_rate): 
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten()) #flatten 2d array in 1d 
  model.add (tf.keras.layers.Dense(units=32, activation='relu')) #define the first hidden layer
  model.add(tf.keras.layers.Dropout(rate=0.2)) #define a dropout regularization layer 
  model.add(tf.keras.layers.Dense(units=10, activation='softmax')) #define the output layer 
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate), loss="sparse_categorical_crossentropy", metrics=['accuracy']) #construct layers into a model that tensorflow can execute 
  return model 

In [ ]:
def train_model(model, train_features, train_label, epochs, batch_size=None, validation_split=0.1): 
  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
  # To track the progression of training, gather a snapshot
  # of the model's metrics at each epoch. 
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  return epochs, hist 

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.003
epochs = 50
batch_size = 4000
validation_split = 0.2

# Establish the model's topography.
my_model = create_model(learning_rate)

# Train the model on the normalized training set.
epochs, hist = train_model(my_model, embeddings_train, y_train, 
                           epochs, batch_size, validation_split)

# Plot a graph of the metric vs. epochs.
list_of_metrics_to_plot = ['accuracy']
#plot_curve(epochs, hist, list_of_metrics_to_plot)

# Evaluate against the test set.
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x=embeddings_test, y=y_test, batch_size=batch_size)